# Making Pandas DataFrames from API Requests
In this example, we will use the U.S. Geological Survey's API to grab a JSON object of earthquake data and convert it to a `pandas.DataFrame`.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

### Get Data from API

In [2]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

200

Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON payload, we can extract it from the response with the `json()` method.

### Isolate the Data from the JSON Response
We need to check the structures of the response data to know where our data is.

In [3]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

The USGS API provides information about our request in the `metadata` key. Note that your result will be different, regardless of the date range you chose, because the API includes a timestamp for when the data was pulled:

In [4]:
earthquake_json['metadata']

{'generated': 1680574448000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2023-03-03&endtime=2023-04-02',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.13.6',
 'count': 12902}

Each element in the JSON array `features` is a row of data for our dataframe.

In [7]:
type(earthquake_json['features'])

list

Your data will be different depending on the date you run this.

In [8]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 1.59,
  'place': '100 km W of Adak, Alaska',
  'time': 1680393281990,
  'updated': 1680562897780,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/av91829861',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=av91829861&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 39,
  'net': 'av',
  'code': '91829861',
  'ids': ',av91829861,',
  'sources': ',av,',
  'types': ',origin,phase-data,',
  'nst': 5,
  'dmin': None,
  'rms': 0.31,
  'gap': 207,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 1.6 - 100 km W of Adak, Alaska'},
 'geometry': {'type': 'Point', 'coordinates': [-178.09, 51.863, 3.32]},
 'id': 'av91829861'}

### Convert to DataFrame
We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.

In [9]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,1.59,"100 km W of Adak, Alaska",1680393281990,1680562897780,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",av91829861,",",av,",",origin,phase-data,",5.0,NaN,0.31,207.0,ml,earthquake,"M 1.6 - 100 km W of Adak, Alaska"
1,1.25,"1km ENE of La Palma, CA",1680393096720,1680457945714,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ci39508858,",",ci,",",nearby-cities,origin,phase-data,scitech-link,",29.0,0.05219,0.17,95.0,ml,earthquake,"M 1.3 - 1km ENE of La Palma, CA"
2,1.51,"14km WNW of Anza, CA",1680392678240,1680536657840,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ci39508850,",",ci,",",focal-mechanism,nearby-cities,origin,phase-da...",53.0,0.04664,0.14,31.0,ml,earthquake,"M 1.5 - 14km WNW of Anza, CA"
3,1.30,None,1680392557833,1680562654610,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",av91829851,ak02346vum0l,",",av,ak,",",origin,phase-data,",NaN,NaN,0.42,NaN,ml,earthquake,M 1.3 -
4,0.05,"2 km W of Unalaska, Alaska",1680392168390,1680562357190,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",av91829836,",",av,",",origin,phase-data,",5.0,NaN,0.07,158.0,ml,earthquake,"M 0.1 - 2 km W of Unalaska, Alaska"


### (Optional) Write Data to CSV

In [7]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./4-inspecting_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>